# Part 2: Data Analytics 
(Raj Dhakan, xn23972)
***
## Importing Libraries

In [8]:
import json
import pandas as pd
import requests
import numpy as np
import time
from sigfig import round

## Step 1: Crawling Real-World Datasets
***
The dataset that is extracted is about S&P500 stocks. S&P500 is a common equity indicies which include 500 of the largest companies listed on stock exchanges in the United States. 

First, the table of S&P500 companies is scarped from Wikipedia's __[S&P500 Companies](https://en.wikipedia.org/wiki/List_of_S%26P_500_companies)__ homepage. The columns of interest from this table are: Symbol of the stock (e.g. AAPL for Apple Inc.), Security (i.e. the company name), Global Industry Classification Standard (GICS, i.e. field of work such as Tech, Finance, Healthcare etc) sector, and Headquarters Location. 

Second, common key metrics used in analysing stocks are scraped from __[Yahoo Finance](https://finance.yahoo.com/)__. The key metrics of interest are: Market Capitilisation, Revenue, Profit Margin, Earning per Share, Profit to Earnings ratio and Profit to Earning Growth ratio (5yr forecasted). These metrics are scraped by taking the symbols from the table acquired from Wikipedia and using them to create a url to the respective stock's statistics page (e.g. __https://finance.yahoo.com/quote/AAPL/key-statistics?p=AAPL__ for Apple Inc.)

The two tables are then merged and saved as `SnP500_raw.csv` in the directory of this jupyter notebook

<div class="alert alert-block alert-info">
<b>Note:</b> The data extracted from Yahoo Finance is accurate to date 08/12/2023.
</div>


In [9]:
# extracting list of SnP 500 companies from Wikipedia
url_link= 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
response = requests.get(url_link)
SnP500_raw = pd.read_html(response.text)[0]
SnP500_raw.head()

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In [10]:
def yahoo_api_statistics(symbol:str):
    '''
    Returns a list of metrics commonly used to evaluate stocks.

            Parameters:
                    symbol (str): Stock symbols; e.g. AAPL for Apple

            Returns:
                    metrics_list (lst): List of key metrics in the given order:
                    1. Market Capitilisation (Market_cap)
                    2. Revenue (Revenue)
                    3. Profit Margin (Profit_margin)
                    4. Earning per Share (EPS)
                    5. Profit to Earnings ratio (PE_ratio)
                    6. Profit to Earnings Growth ratio (PEG)
    '''  
    # replaces a '.' to '-' in any symbol to produce the correct url to scrape data
    if "." in symbol:
        symbol = symbol.replace(".","-")
        
    statistics_url = ("https://finance.yahoo.com/quote/{symbol}/key-statistics?p={symbol}").format(symbol = symbol)
    response = requests.get(statistics_url, headers= {'User-agent': 'Mozilla/5.0'})
    df = pd.read_html(response.text)
    
    # Extracts individual metrics from the tables (use of iat over iloc to increase speed of extraction)
    Market_cap = df[0].iat[0,1]
    Revenue = df[-3].iat[0,1]
    Profit_margin = df[5].iat[0,1]
    EPS = df[-3].iat[6,1]
    PE_ratio =  df[0].iat[2,1]
    PEG = df[0].iat[4,1]
    
    metrics_list = [Market_cap, Revenue, Profit_margin, EPS, PE_ratio, PEG]
    
    return metrics_list


# creating an empty list to store the metrics
metrics = []

# loops through symbols to extract 
for i in range(len(SnP500_raw)):
    
    # try/except to catch out any pages with errors
    try:
        symbol = SnP500_raw.iat[i,0]
        input_list = yahoo_api_statistics(symbol)
        metrics.append(input_list)
    except:
        print(symbol)


In [11]:
# columns with units, where B = Billions and TTM = Trailing Twelve Months
columns_to_add = ['Market Cap / B', 'Revenue (TTM) / B', 'Profit Margin / %', 'Earnings per Share (TTM) / $', 'Price to Earning ratio (TTM)', 'Price to Earnings Growth ratio (5yr expected)']

# converting lists of lists into a dataframe
metrics = pd.DataFrame(data = metrics, columns = columns_to_add)

# slicing the raw dataframe to acquire the required columns only 
SnP500_sliced = SnP500_raw[['Symbol','Security','GICS Sector', 'Headquarters Location']]

# merging the two dataframes
SnP500_metrics_raw = pd.concat([SnP500_sliced,metrics], axis = 1)

# saving the merged dataframe as csv file in the directory of this Jupyter Notebook
SnP500_metrics_raw.to_csv("SnP500_raw_data.csv", index= False)

## Step 2: Data Preparation & Cleaning
***
The following is performed to prepare the data:
- Units of the key metrics are standardised for all stocks and removed from each cell (e.g. for Market Capitalisation any value in T (Trillions) is converted to B, and 302.1B is changed to 302.1 since units are stated on the column headings) ---done
- Null values are indentified and either rows are removed or values are filled via manual calculations
- List of companies are sorted in the order of largest Market Capitalisation to the smallest and a ranking index is produced --- done
- Ensuring metrics are to 4 significant figures --- done
- Column headings are simplified where possible (e.g. Security is changed to Company) --- done
- Headquarters location of Security (or Companies) are standardised (i.e. the state/city is separated in another column to the country) --- done

In [13]:
# reading csv file
SnP500_raw_file_data = pd.read_csv("SnP500_raw_data.csv")
SnP500_raw_file_data.head()

,Symbol,Security,GICS Sector,Headquarters Location,Market Cap / B,Revenue (TTM) / B,Profit Margin / %,Earnings per Share (TTM) / $,Price to Earning ratio (TTM),Price to Earnings Growth ratio (5yr expected)
0,MMM,3M,Industrials,"Saint Paul, Minnesota",57.04B,32.75B,-22.59%,-13.31,16.99,3.79
1,AOS,A. O. Smith,Industrials,"Milwaukee, Wisconsin",11.60B,3.8B,7.87%,2.01,38.81,2.00
2,ABT,Abbott,Health Care,"North Chicago, Illinois",180.64B,39.96B,12.92%,2.94,35.39,24.94
3,ABBV,AbbVie,Health Care,"North Chicago, Illinois",261.25B,55.14B,11.81%,3.65,40.54,3.61
4,ACN,Accenture,Information Technology,"Dublin, Ireland",210.31B,64.11B,10.72%,10.76,31.11,2.74


In [17]:
# simplify column names
SnP500_columns_renamed = SnP500_raw_file_data.copy()
SnP500_columns_renamed = SnP500_columns_renamed.rename(columns={"Security": "Company", "Headquarters Location": "HQ Country"})
SnP500_columns_renamed.head()

,Symbol,Company,GICS Sector,HQ Country,Market Cap / B,Revenue (TTM) / B,Profit Margin / %,Earnings per Share (TTM) / $,Price to Earning ratio (TTM),Price to Earnings Growth ratio (5yr expected)
0,MMM,3M,Industrials,"Saint Paul, Minnesota",57.04B,32.75B,-22.59%,-13.31,16.99,3.79
1,AOS,A. O. Smith,Industrials,"Milwaukee, Wisconsin",11.60B,3.8B,7.87%,2.01,38.81,2.00
2,ABT,Abbott,Health Care,"North Chicago, Illinois",180.64B,39.96B,12.92%,2.94,35.39,24.94
3,ABBV,AbbVie,Health Care,"North Chicago, Illinois",261.25B,55.14B,11.81%,3.65,40.54,3.61
4,ACN,Accenture,Information Technology,"Dublin, Ireland",210.31B,64.11B,10.72%,10.76,31.11,2.74


In [18]:
# identify which columns have nan values and how many
print(SnP500_columns_renamed.isna().sum())

# as such columns 'Price to Earning ratio' and 'Price to Earnings Growth ratio' will be cleaned last

Symbol                                            0
Company                                           0
GICS Sector                                       0
HQ Country                                        0
Market Cap / B                                    0
Revenue (TTM) / B                                 0
Profit Margin / %                                 0
Earnings per Share (TTM) / $                      0
Price to Earning ratio (TTM)                     20
Price to Earnings Growth ratio (5yr expected)    94
dtype: int64


In [19]:
# standardising HQ country column to contain countries only
url_link= 'https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States'
response = requests.get(url_link)
american_states = pd.read_html(response.text)[1].iloc[:,0]


# # cleaning data fetched from Wikipedia
american_states = list(american_states.str.rstrip(to_strip="[B]"))
print(american_states)

# indentifying unique countries/states, and duplication
unique_states_countries = set([location.split(',')[-1].strip() for location in list(SnP500_columns_renamed.iloc[:,3])])  
print(unique_states_countries)

# creating a copy of dataframe where HQ location data can be cleaned 
SnP500_HQ_location_cleaned = SnP500_columns_renamed.copy()

# creating empty list to store the states or city extracted from the HQ location column
state_or_city = []

for i in range(len(SnP500_columns_renamed)):
    
    location = (SnP500_columns_renamed.iat[i,3]).split(',')[-1].strip()
    
    if (location in american_states) or (location == "D.C."):
        
        SnP500_HQ_location_cleaned.iat[i,3] = "United States"
    
        if location == "D.C.":
            state_or_city.append("Washington")
        else:
            state_or_city.append(location)
            
    elif location == "UK":
        
        SnP500_HQ_location_cleaned.iat[i,3] = "United Kingdom"
        state_or_city.append(((SnP500_columns_renamed.iat[i,3]).split(',')[-2]).strip())
    
    else:
        
        SnP500_HQ_location_cleaned.iat[i,3] = location
        state_or_city.append(((SnP500_columns_renamed.iat[i,3]).split(',')[-2]).strip())

state_or_city = pd.DataFrame(data = state_or_city)

SnP500_HQ_location_cleaned.insert(loc = 4, value = state_or_city, column = "HQ State/City")

SnP500_HQ_location_cleaned.to_csv("tests/test1_HQ_Location.csv", index= False)

['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']
{'Iowa', 'Virginia', 'New Jersey', 'New York', 'Switzerland', 'Oregon', 'Indiana', 'Israel', 'Nevada', 'Utah', 'UK', 'Florida', 'Alabama', 'D.C.', 'Rhode Island', 'Illinois', 'Arizona', 'Colorado', 'Netherlands', 'Massachusetts', 'Louisiana', 'Wisconsin', 'Montana', 'Bermuda', 'North Carolina', 'Ireland', 'Oklahoma', 'California', 'Canada', 'Tennessee', 'Washington', 'Georgi

In [20]:
SnP500_HQ_location_cleaned.head()

,Symbol,Company,GICS Sector,HQ Country,HQ State/City,Market Cap / B,Revenue (TTM) / B,Profit Margin / %,Earnings per Share (TTM) / $,Price to Earning ratio (TTM),Price to Earnings Growth ratio (5yr expected)
0,MMM,3M,Industrials,United States,Minnesota,57.04B,32.75B,-22.59%,-13.31,16.99,3.79
1,AOS,A. O. Smith,Industrials,United States,Wisconsin,11.60B,3.8B,7.87%,2.01,38.81,2.00
2,ABT,Abbott,Health Care,United States,Illinois,180.64B,39.96B,12.92%,2.94,35.39,24.94
3,ABBV,AbbVie,Health Care,United States,Illinois,261.25B,55.14B,11.81%,3.65,40.54,3.61
4,ACN,Accenture,Information Technology,Ireland,Dublin,210.31B,64.11B,10.72%,10.76,31.11,2.74


In [35]:
# creating a copy of dataframe where values for 'Market Cap', 'Revenue' and Profit Margin' can be cleaned 
SnP500_units_cleaned = SnP500_HQ_location_cleaned.copy()

# checking for different units in columns Market Cap and Revenue
units_market_cap = set(SnP500_units_cleaned['Market Cap / B'].str.slice(-1))
units_revenue = set(SnP500_units_cleaned['Revenue (TTM) / B'].str.slice(-1))
units_profit_margin = set(SnP500_units_cleaned['Profit Margin / %'].str.slice(-1))
print(units_market_cap)
print(units_revenue)
print(units_profit_margin)

# based on this all four columns need to have the units removed and rounded to 2 decimal places,
# and 'Market Cap' and 'Revenue' need to have units standardised to Billions

for i in range(len(SnP500_units_cleaned)):
    
    SnP500_units_cleaned.iat[i,7] = round(float(SnP500_units_cleaned.iat[i,7][:-1]), decimals=2)
    
    if SnP500_units_cleaned.iat[i,5][-1]=="T":
        SnP500_units_cleaned.iat[i,5] = round(float(SnP500_units_cleaned.iat[i,5][:-1])*1000, decimals=2)
    else:
        SnP500_units_cleaned.iat[i,5] = round(float(SnP500_units_cleaned.iat[i,5][:-1]), decimals=2)
        
    
    if SnP500_units_cleaned.iat[i,6][-1]=="M":
        SnP500_units_cleaned.iat[i,6] = round(float(SnP500_units_cleaned.iat[i,6][:-1])/1000, decimals=2)
    else:
        SnP500_units_cleaned.iat[i,6] = round(float(SnP500_units_cleaned.iat[i,6][:-1]), decimals=2)

SnP500_units_cleaned.to_csv("tests/test2_Standardising_Units.csv", index= False)

{'B', 'T'}
{'B', 'M'}
{'%'}


In [23]:
SnP500_units_cleaned.head()

,Symbol,Company,GICS Sector,HQ Country,HQ State/City,Market Cap / B,Revenue (TTM) / B,Profit Margin / %,Earnings per Share (TTM) / $,Price to Earning ratio (TTM),Price to Earnings Growth ratio (5yr expected)
0,MMM,3M,Industrials,United States,Minnesota,57.04,32.75,-22.59,-13.31,16.99,3.79
1,AOS,A. O. Smith,Industrials,United States,Wisconsin,11.6,3.8,7.87,2.01,38.81,2.00
2,ABT,Abbott,Health Care,United States,Illinois,180.64,39.96,12.92,2.94,35.39,24.94
3,ABBV,AbbVie,Health Care,United States,Illinois,261.25,55.14,11.81,3.65,40.54,3.61
4,ACN,Accenture,Information Technology,Ireland,Dublin,210.31,64.11,10.72,10.76,31.11,2.74


In [24]:
SnP500_sorted = SnP500_units_cleaned.copy()

# ordering the company lists based on 'Market Cap'
SnP500_sorted = SnP500_sorted.sort_values("Market Cap / B", ascending = False, ignore_index=True)
SnP500_sorted.head()

,Symbol,Company,GICS Sector,HQ Country,HQ State/City,Market Cap / B,Revenue (TTM) / B,Profit Margin / %,Earnings per Share (TTM) / $,Price to Earning ratio (TTM),Price to Earnings Growth ratio (5yr expected)
0,AAPL,Apple Inc.,Information Technology,United States,California,3020.0,383.29,25.31,6.14,31.69,2.33
1,MSFT,Microsoft,Information Technology,United States,Washington,2760.0,218.31,35.31,10.31,35.94,2.21
2,GOOG,Alphabet Inc. (Class C),Communication Services,United States,California,1720.0,297.13,22.46,5.22,26.52,1.30
3,GOOGL,Alphabet Inc. (Class A),Communication Services,United States,California,1720.0,297.13,22.46,5.22,26.24,1.29
4,AMZN,Amazon,Consumer Discretionary,United States,Washington,1520.0,554.03,3.62,1.91,76.90,2.42


In [25]:
SnP500_sorted.to_csv("tests/test3_Market_Cap_Ordering.csv", index= False)

In [26]:
# ensuring data quality maintained by checking on null values
print(SnP500_sorted.isna().sum())

Symbol                                            0
Company                                           0
GICS Sector                                       0
HQ Country                                        0
HQ State/City                                     0
Market Cap / B                                    0
Revenue (TTM) / B                                 0
Profit Margin / %                                 0
Earnings per Share (TTM) / $                      0
Price to Earning ratio (TTM)                     20
Price to Earnings Growth ratio (5yr expected)    94
dtype: int64


In [30]:
SnP500_PEG_dropped = SnP500_sorted.drop(labels="Price to Earnings Growth ratio (5yr expected)", axis = 1)
SnP500_PEG_dropped.to_csv("tests/test4_PEG_column_dropped.csv", index= False)

In [31]:
SnP500_PEG_dropped.head()

,Symbol,Company,GICS Sector,HQ Country,HQ State/City,Market Cap / B,Revenue (TTM) / B,Profit Margin / %,Earnings per Share (TTM) / $,Price to Earning ratio (TTM)
0,AAPL,Apple Inc.,Information Technology,United States,California,3020.0,383.29,25.31,6.14,31.69
1,MSFT,Microsoft,Information Technology,United States,Washington,2760.0,218.31,35.31,10.31,35.94
2,GOOG,Alphabet Inc. (Class C),Communication Services,United States,California,1720.0,297.13,22.46,5.22,26.52
3,GOOGL,Alphabet Inc. (Class A),Communication Services,United States,California,1720.0,297.13,22.46,5.22,26.24
4,AMZN,Amazon,Consumer Discretionary,United States,Washington,1520.0,554.03,3.62,1.91,76.90


## Step 3: Exploratory Analysis
***

PE = current share price / (sum of EPS over the last 4 quarters)
needs to have negative PE_ratio where there is negative values

## Step 4: Investigating Data-Set with questions
***

## Step 5: Conclusion
***